# Libraries

In [ ]:
# generic imports
import pandas as pd
import numpy as np
import datetime

# data augmentation imports
from be_great import GReaT # https://github.com/kathrinse/be_great

: 

## Load the Dataset

In [3]:
# Import DNN-EdgeIIoT-dataset.csv file from Kaggle (https://www.kaggle.com/datasets/mohamedamineferrag/edgeiiotset-cyber-security-dataset-of-iot-iiot)
df = pd.read_csv('../data/DNN-EdgeIIoT-dataset.csv')

/tmp/ipykernel_907213/1440379499.py:2: DtypeWarning: Columns (2,3,6,11,13,14,15,16,17,31,32,34,39,45,51,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/DNN-EdgeIIoT-dataset.csv')


#### AQUI DEVEM ESTAR AS TRANSFORMAÇÔES QUE O DATASET SOFREU ATÉ EdgeIIot_train_100k.csv

In [5]:
# Load data relative to the training set, with Attack_type "Normal" with 100k rows (SHORCUT)
df_train = pd.read_csv("../data/EdgeIIot_train_100k.csv", low_memory=False)

In [6]:
df_train.head()

,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,icmp.unused,http.content_length,http.request.method,http.referer,http.request.version,http.response,...,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id,Attack_label,Attack_type
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DDoS_UDP
1,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Password
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DDoS_UDP
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DDoS_UDP
4,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Password


# Data Augmentation GReaT

## Model Training

In [8]:
# instantiate the GReaT model with the distilgpt2 language model
model = GReaT(llm='distilgpt2', 
              batch_size=32, 
              epochs=25)

great = GReaT(llm="distilgpt2",              # Name of the large language model used (see HuggingFace for more options)
              epochs=10,                     # Number of epochs to train
              save_steps=2000,               # Save model weights every x steps
              logging_steps=500,             # Log the loss and learning rate every x steps
              experiment_dir="checkpoints/GReat", # Name of the directory where all intermediate steps are saved
              batch_size=16,                 # Batch Size
              #lr_scheduler_type="constant", # Specify the learning rate scheduler 
              #learning_rate=5e-5            # Set the inital learning rate
             )


# fit the model on the training data
model.fit(df_train)

## Train the Model

In [186]:
# instantiate the DecisionTreeClassifier model (using the default parameters)
clf = tree.DecisionTreeClassifier()
# Train the model on training data
dtree = clf.fit(X_train_sm, y_train_sm)

## Save the model

In [ ]:
great.save("california")

## Generates preditions on the Test data

In [187]:
predictions = dtree.predict(X_test)

# Evaluate the model

In [188]:
# calculate accuracy, precision, recall, f1-score, and kappa score
accuracy = metrics.accuracy_score(y_test, predictions)
precision = metrics.precision_score(y_test, predictions, average='weighted')
recall = metrics.recall_score(y_test, predictions, average='weighted')
f1score = metrics.f1_score(y_test, predictions, average='weighted')
kappa = metrics.cohen_kappa_score(y_test, predictions)

In [189]:
# print the results in a nice table with metrics. Values are rounded to 4 decimal places and aligned right
print("Decision Tree with SMOTE Augmentation") 
print("====================")
print("Accuracy:" + str(round(accuracy, 4)).rjust(10))
print("Precision:" + str(round(precision, 4)).rjust(9))
print("Recall:" + str(round(recall, 4)).rjust(12))
print("F1-score:" + str(round(f1score, 4)).rjust(10))
print("Kappa:" + str(round(kappa, 4)).rjust(13))
print("====================")

Decision Tree with SMOTE Augmentation
Accuracy:    0.9796
Precision:   0.9807
Recall:      0.9796
F1-score:      0.98
Kappa:       0.9575


In [190]:
# Confusion matrix
conf_mat = metrics.confusion_matrix(y_test, predictions)
conf_mat = pd.DataFrame(conf_mat)
conf_mat.index.name = 'Actual'
conf_mat.columns.name = 'Predicted'
print(conf_mat)

Predicted    0     1      2      3      4    5   6       7     8     9     10  \
Actual                                                                          
0          4527     0      0      0      0   90   0       0     0    61   104   
1             0  8390      0      0      0    0   0       0     0     0     0   
2             0     0  13474      0      0   27   0       0     0     0     0   
3             0     0      0  10009      0    0   0       0     0     0     0   
4             0     0      0      0  24601    0   0       0     0     0     0   
5             5     0      0      0      0  116   0       0     0    15    10   
6             0     0      0      0      0    0  76       0     0     0     0   
7             0     0      0      0      0    0   0  272776     0     0     0   
8             0     0      0      0      0    0   0       0  8351     0     0   
9           103     0      0      0      0  226   0       0     0  3598   135   
10           53     0      0

## Save results to file

In [167]:
results = {
    "model": "Decision Tree",
    "augmentation": "SMOTE",
    "dataset": "EdgeIIoT",
    "accuracy": accuracy,
    "precision": precision,
    "recall": recall,
    "f1score": f1score,
    "kappa": kappa,
    "timestamp": str(datetime.datetime.now())}

# Save results to CSV file to results path in results.csv file
save_results_to_csv([results], config['results_path'])

# # First we add a line with the modelname, augmentation, dataset, timestamp to conf_mat dataframe on the top line
# conf_mat = conf_mat.stack().rename_axis(('Actual', 'Predicted')).reset_index(name='Amount')
# conf_mat.insert(0, 'Model', 'Decision Tree')
# conf_mat.insert(1, 'Augmentation', 'SMOTE')
# conf_mat.insert(2, 'Dataset', 'EdgeIIoT')
# conf_mat.insert(3, 'Timestamp', str(datetime.datetime.now()))
# # then we append the results to the conf_mat.csv file
# conf_mat.to_csv(config['conf_matrix_path'], mode='a', header=False, index=False)

In [196]:
# Create a DataFrame for the metadata
metadata = pd.DataFrame({
    'Model Name': ['Decision Tree'],
    'Timestamp': [str(datetime.datetime.now())],
    'Augmentation': ['SMOTE']
})

# Save the metadata and the confusion matrix to a CSV file
metadata.to_csv(config['conf_matrix_path'], mode='a', index=False)
# take the confusion matrix structure and append it to the csv file
conf_mat.to_csv(config['conf_matrix_path'], mode='a', header=True, index=True)